In [1]:
%pip install feedparser
%pip install tqdm
import pandas as pd
import numpy as np
import yfinance as yf
import feedparser
from transformers import pipeline
from datetime import datetime, timedelta
from tabulate import tabulate
from tqdm import tqdm
from google.colab import userdata
userdata.get('Read_Hugging_Face_Hub')

# --- Configuration ---
tickers = ['GFL.to', 'CCO.to', 'HBM.to', 'CIGI.to', 'AG.to', 'AGI.to', 'BN.to', 'SVM.to', 'FSV.to', 'STN.to', 'SSL.to', 'K.to', 'WPM.to', 'WCN.to', 'OR.to', 'LUN.to', 'CP.to', 'FNV.to', 'TFPM.to', 'RBA.to', 'TRI.to', 'AEM.to', 'DOO.to', 'EFX.to', 'PAAS.to', 'NGT.to', 'TA.to', 'GIL.to', 'SII.to', 'ABX.to', 'MAG.to', 'WFG.to', 'TFII.to', 'BTO.to', 'MX.to', 'LUG.to', 'NOA.to', 'IMO.to', 'CNR.to', 'CG.to', 'BAM.to', 'OVV.to', 'BTE.to', 'RY.to', 'ASTL.to', 'FTS.to', 'QSR.to', 'EDV.to', 'OTEX.to', 'NTR.to', 'CVE.to', 'CM.to', 'MFC.to', 'TD.to', 'SLF.to', 'SU.to', 'BMO.to', 'BIPC.to', 'BEPC.to', 'AQN.to', 'EMA.to', 'MG.to', 'VET.to', 'TRP.to', 'ORA.to', 'CIA.to', 'BCE.to', 'CNQ.to', 'PPL.to', 'BNS.to', 'ENB.to', 'MER.to', 'T.to', 'TAL.to']
news_days = 3
data_days = 365 * 3
start_date = (datetime.now() - timedelta(days=data_days)).strftime('%Y-%m-%d')
end_date = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')

stoc_p = 13
stoc_sp = 3
rsi_p = 13
bb_win = 30
bb_std = 2
mom_p = 10
bbp_p = 13
obv_ewm = 20
vpt_ewm = 29
short_ewm = 5
mid_ewm = 8
long_ewm = 13
macd_p1 = 12
macd_p2 = 26
macd_sl = 9

#weights
news_today = 2.0
news_yesterday = 1.5
news_2_days_ago = 1.25
ema_w = 1.2
macd_w = 1.0
stoc_w = 0.8
rsi_w = 0.7
bb_w = 1.0
mom_w = 0.6
bbp_w = 0.9
obv_w = 1.1
vpt_w = 1.1 # Proper weight for Volume Price Trend

top_n =10

pipe = pipeline(task="text-classification", model="ProsusAI/finbert")
all_results = []
all_sentiment_results = []

# --- Indicator Calculation and Signal Generation Functions ---

def get_signal(buy_condition, sell_condition):
    if buy_condition:
        return 'BUY'
    elif sell_condition:
        return 'SELL'
    else:
        return '---'

def get_ema_signal(data):
    signal_list = ['---'] * len(data)
    for i in range(1, len(data)):
        buy_condition = (data['Short'].iloc[i] > data['Middle'].iloc[i] and
                         data['Middle'].iloc[i] > data['Long'].iloc[i] and
                         data['Short'].iloc[i-1] <= data['Middle'].iloc[i-1])

        sell_condition = (data['Short'].iloc[i] < data['Middle'].iloc[i] and
                          data['Middle'].iloc[i] < data['Long'].iloc[i] and
                          data['Short'].iloc[i-1] >= data['Middle'].iloc[i-1])

        signal_list[i] = get_signal(buy_condition, sell_condition)
    return pd.Series(signal_list, index=data.index)

def get_macd_signal(df):
    signal_list = ['---'] * len(df)
    for i in range(1, len(df)):
        if pd.isna(df['MACD'].iloc[i]) or pd.isna(df['Signal Line'].iloc[i]):
            continue

        buy_condition = (df['MACD'].iloc[i] > df['Signal Line'].iloc[i] and
                         df['MACD'].iloc[i-1] <= df['Signal Line'].iloc[i-1])

        sell_condition = (df['MACD'].iloc[i] < df['Signal Line'].iloc[i] and
                          df['MACD'].iloc[i-1] >= df['Signal Line'].iloc[i-1])

        signal_list[i] = get_signal(buy_condition, sell_condition)
    return pd.Series(signal_list, index=df.index)

def calculate_stochastic_oscillator(df: pd.DataFrame, period: int = stoc_p, signal_period: int = stoc_sp) -> pd.DataFrame:
    df_copy = df.copy()
    df_copy['Lowest_Low'] = df_copy['Low'].rolling(window=period).min()
    df_copy['Highest_High'] = df_copy['High'].rolling(window=period).max()
    denominator = (df_copy['Highest_High'] - df_copy['Lowest_Low']).replace(0, np.nan)
    numerator = (df_copy['Close'] - df_copy['Lowest_Low'])
    percent_k_raw = 100 * (numerator / denominator)
    df_copy['%K'] = percent_k_raw.ffill().fillna(50)
    df_copy['%D'] = df_copy['%K'].rolling(window=signal_period).mean()
    df_copy['Stoch_Signal'] = '---'
    df_copy.loc[(df_copy['%K'].shift(1) < df_copy['%D'].shift(1)) &
                (df_copy['%K'] > df_copy['%D']) &
                (df_copy['%K'] < 20), 'Stoch_Signal'] = 'BUY'
    df_copy.loc[(df_copy['%K'].shift(1) > df_copy['%D'].shift(1)) &
                (df_copy['%K'] < df_copy['%D']) &
                (df_copy['%K'] > 80), 'Stoch_Signal'] = 'SELL'
    return df_copy

def calculate_rsi(df: pd.DataFrame, period: int = rsi_p) -> pd.DataFrame:
    df_copy = df.copy()
    delta = df_copy['Close'].diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.ewm(com=period - 1, adjust=False).mean()
    avg_loss = loss.ewm(com=period - 1, adjust=False).mean()
    rs = avg_gain / avg_loss
    df_copy['RSI'] = 100 - (100 / (1 + rs))
    df_copy['RSI_Signal'] = '---'
    df_copy.loc[(df_copy['RSI'].shift(1) <= 40) & (df_copy['RSI'] > 40), 'RSI_Signal'] = 'BUY'
    df_copy.loc[(df_copy['RSI'].shift(1) >= 60) & (df_copy['RSI'] < 60), 'RSI_Signal'] = 'SELL'
    return df_copy

def calculate_bollinger_bands(df: pd.DataFrame, window: int = bb_win, num_std_dev: int = bb_std) -> pd.DataFrame:
    df_copy = df.copy()
    df_copy['Middle_Band'] = df_copy['Close'].rolling(window=window).mean()
    df_copy['Std_Dev'] = df_copy['Close'].rolling(window=window).std()
    df_copy['Upper_Band'] = df_copy['Middle_Band'] + (df_copy['Std_Dev'] * num_std_dev)
    df_copy['Lower_Band'] = df_copy['Middle_Band'] - (df_copy['Std_Dev'] * num_std_dev)
    df_copy['BB_Signal'] = '---'
    df_copy.loc[df_copy['Close'] < df_copy['Lower_Band'], 'BB_Signal'] = 'BUY'
    df_copy.loc[df_copy['Close'] > df_copy['Upper_Band'], 'BB_Signal'] = 'SELL'
    return df_copy

def calculate_momentum_signals(df: pd.DataFrame, period: int = mom_p) -> pd.DataFrame:
    df_copy = df.copy()
    df_copy['Momentum_Oscillator'] = (df_copy['Close'] / df_copy['Close'].shift(period)) * 100
    df_copy['Momentum_Signal'] = '---'
    df_copy.loc[(df_copy['Momentum_Oscillator'].shift(1) <= 100) & (df_copy['Momentum_Oscillator'] > 100), 'Momentum_Signal'] = 'BUY'
    df_copy.loc[(df_copy['Momentum_Oscillator'].shift(1) >= 100) & (df_copy['Momentum_Oscillator'] < 100), 'Momentum_Signal'] = 'SELL'
    return df_copy

def calculate_bull_bear_power(df: pd.DataFrame, ema_period: int = bbp_p) -> pd.DataFrame:
    df_copy = df.copy()
    df_copy['EMA_BBP'] = df_copy['Close'].ewm(span=ema_period, adjust=False).mean()
    df_copy['Bull_Power'] = df_copy['High'] - df_copy['EMA_BBP']
    df_copy['Bear_Power'] = df_copy['Low'] - df_copy['EMA_BBP']
    df_copy['BBP_Signal'] = '---'
    buy_conditions = (df_copy['EMA_BBP'].diff() > 0) & \
                     (df_copy['Bear_Power'] < 0) & \
                     (df_copy['Bear_Power'].diff() > 0)
    df_copy.loc[buy_conditions, 'BBP_Signal'] = 'BUY'
    sell_conditions = (df_copy['EMA_BBP'].diff() < 0) & \
                      (df_copy['Bull_Power'] > 0) & \
                      (df_copy['Bull_Power'].diff() < 0)
    df_copy.loc[sell_conditions, 'BBP_Signal'] = 'SELL'
    return df_copy

def calculate_obv(df: pd.DataFrame) -> pd.DataFrame:
    df_copy = df.copy()
    df_copy['OBV'] = np.where(df_copy['Close'] > df_copy['Close'].shift(1), df_copy['Volume'],
                              np.where(df_copy['Close'] < df_copy['Close'].shift(1), -df_copy['Volume'], 0)).cumsum()
    df_copy['OBV_EMA'] = df_copy['OBV'].ewm(span=obv_ewm, adjust=False).mean()  # 20-period EMA of OBV for signal line
    df_copy['OBV_Signal'] = '---'
    buy_conditions = (df_copy['OBV'] > df_copy['OBV_EMA']) & (df_copy['OBV'].shift(1) <= df_copy['OBV_EMA'].shift(1))
    sell_conditions = (df_copy['OBV'] < df_copy['OBV_EMA']) & (df_copy['OBV'].shift(1) >= df_copy['OBV_EMA'].shift(1))
    df_copy.loc[buy_conditions, 'OBV_Signal'] = 'BUY'
    df_copy.loc[sell_conditions, 'OBV_Signal'] = 'SELL'
    return df_copy

def calculate_vpt(df: pd.DataFrame) -> pd.DataFrame:
    df_copy = df.copy()
    df_copy['VPT'] = (df_copy['Close'].diff() / df_copy['Close'].shift(1)) * df_copy['Volume']
    df_copy['VPT'] = df_copy['VPT'].cumsum()
    df_copy['VPT_EMA'] = df_copy['VPT'].ewm(span=vpt_ewm, adjust=False).mean()
    df_copy['VPT_Signal'] = '---'
    buy_conditions = (df_copy['VPT'] > df_copy['VPT_EMA']) & (df_copy['VPT'].shift(1) <= df_copy['VPT_EMA'].shift(1))
    sell_conditions = (df_copy['VPT'] < df_copy['VPT_EMA']) & (df_copy['VPT'].shift(1) >= df_copy['VPT_EMA'].shift(1))
    df_copy.loc[buy_conditions, 'VPT_Signal'] = 'BUY'
    df_copy.loc[sell_conditions, 'VPT_Signal'] = 'SELL'
    return df_copy


def calculate_traditional_pivots(high, low, close):
    pivot = (high + low + close) / 3
    r1 = (2 * pivot) - low
    s1 = (2 * pivot) - high
    r2 = pivot + (high - low)
    s2 = pivot - (high - low)
    r3 = high + (2 * (pivot - low))
    s3 = low - (2 * (high - pivot))
    return pivot, r1, s1, r2, s2, r3, s3

# --- Step 1: Sentiment Analysis ---

sentiment_results = []
for ticker_full in tqdm(tickers, desc="Processing Sentiment", unit="stock", ncols =100, bar_format='{l_bar}{bar}|{n_fmt}/{total_fmt} [{elapsed} <{remaining},{rate_fmt} {postfix}]', colour='green'):
    ticker = ticker_full
    keyword = ticker_full.replace('.to', '')
    rss_url = f'https://finance.yahoo.com/rss/headline?s={ticker}'
    feed = feedparser.parse(rss_url)
    total_score = 0
    num_articles = 0
    now = datetime.now()
    days_ago = now - timedelta(days=news_days)

    #print(f"\n--- Processing sentiment for {ticker_full} ---")

    for entry in feed.entries:
        if not hasattr(entry, 'published_parsed'):
            continue
        published_date = datetime(*entry.published_parsed[:6])
        if published_date < days_ago:
            continue
        if keyword.lower() not in entry.summary.lower():
            continue

        # Calculate the number of days between now and the published date
        days_diff = (now - published_date).days

        # Determine the multiplier based on the date difference
        multiplier = 1
        if days_diff <= 0:
            multiplier = news_today
        elif days_diff <= 1:
            multiplier = news_yesterday
        elif days_diff <= 2:
            multiplier = news_2_days_ago

        sentiment = pipe(entry.summary)[0]

        # Apply the multiplier to the sentiment score
        weighted_score = sentiment['score'] * multiplier

        if sentiment['label'] == 'positive':
            total_score += weighted_score
            num_articles += 1
        elif sentiment['label'] == 'negative':
            total_score -= weighted_score
            num_articles += 1

    final_score = total_score / num_articles if num_articles > 0 else 0
    final_score = round(final_score, 2)

    s_label = "Neutral"
    if final_score >= 0.15:
        s_label = "Positive"
    elif final_score <= -0.15:
        s_label = "Negative"

    sentiment_results.append({
        'ticker': ticker_full,
        'final_score': final_score,
        's_label': s_label
    })

# Convert sentiment results to a DataFrame for easy lookup
sentiment_df = pd.DataFrame(sentiment_results).set_index('ticker')

# --- Step 2: Technical Analysis and Final Report Generation ---

all_signal_summaries = []
for ticker_full in tqdm(tickers, desc="Processing Technical Data", unit="stock", ncols=100, bar_format='{l_bar}{bar}|{n_fmt}/{total_fmt} [{elapsed} <{remaining}, {rate_fmt} {postfix}]', colour='blue'):
    #print(f"Processing data for {ticker_symbol}...")
    try:
        stock_data = yf.download(ticker_full, start=start_date, end=end_date, actions=False, progress=False, auto_adjust=True)

        if isinstance(stock_data.columns, pd.MultiIndex):
            new_columns = {}
            for level1, level2 in stock_data.columns:
                if level1 in ['Close', 'Adj Close']: new_columns[(level1, level2)] = 'Close'
                elif level1 == 'High': new_columns[(level1, level2)] = 'High'
                elif level1 == 'Low': new_columns[(level1, level2)] = 'Low'
                elif level1 == 'Open': new_columns[(level1, level2)] = 'Open'
                elif level1 == 'Volume': new_columns[(level1, level2)] = 'Volume'
                else: new_columns[(level1, level2)] = '_'.join(col).strip()
            stock_data.columns = pd.Index([new_columns.get(col, col) if isinstance(col, tuple) else col for col in stock_data.columns])

        if stock_data.empty or len(stock_data) < 2:
            print(f"Not enough data for {ticker_full}. Skipping.")
            continue

        if 'Adj Close' in stock_data.columns:
            stock_data['Close'] = stock_data['Adj Close']

        df = stock_data[['Open', 'High', 'Low', 'Close', 'Volume']].copy()
        df.index.name = 'Date'
        df.dropna(inplace=True)

        min_data_points_required = 40
        if len(df) < min_data_points_required:
            print(f"Not enough data points ({len(df)}) for {ticker_full}. Skipping.")
            continue

        numeric_cols_to_round = ['Open', 'High', 'Low', 'Close']
        df[numeric_cols_to_round] = df[numeric_cols_to_round].round(2)

        # Calculate all indicators
        df['Short'] = df.Close.ewm(span=short_ewm, adjust=False).mean().round(2)
        df['Middle']= df.Close.ewm(span=mid_ewm, adjust=False).mean().round(2)
        df['Long']= df.Close.ewm(span=long_ewm, adjust=False).mean().round(2)
        df['MACD'] = df['Close'].ewm(span=macd_p1, adjust=False).mean() - df['Close'].ewm(span=macd_p2, adjust=False).mean()
        df['Signal Line'] = df['MACD'].ewm(span=macd_sl, adjust=False).mean()
        df = calculate_stochastic_oscillator(df)
        df = calculate_rsi(df)
        df = calculate_bollinger_bands(df)
        df = calculate_momentum_signals(df)
        df = calculate_bull_bear_power(df)
        df = calculate_obv(df)
        df = calculate_vpt(df) # Calculate Volume Price Trend

        # Get the last signal from each indicator
        last_ema_signal = get_ema_signal(df).iloc[-1]
        last_macd_signal = get_macd_signal(df).iloc[-1]
        last_stoch_signal = df['Stoch_Signal'].iloc[-1]
        last_rsi_signal = df['RSI_Signal'].iloc[-1]
        last_bb_signal = df['BB_Signal'].iloc[-1]
        last_momentum_signal = df['Momentum_Signal'].iloc[-1]
        last_bbp_signal = df['BBP_Signal'].iloc[-1]
        last_obv_signal = df['OBV_Signal'].iloc[-1]
        last_vpt_signal = df['VPT_Signal'].iloc[-1] # Get last VPT signal
        last_price = df['Close'].iloc[-1]

        # Calculate Pivot Points
        if len(df) >= 2:
            prev_day_high = df['High'].iloc[-2]
            prev_day_low = df['Low'].iloc[-2]
            prev_day_close = df['Close'].iloc[-2]
            pivot, r1, s1, r2, s2, r3, s3 = calculate_traditional_pivots(prev_day_high, prev_day_low, prev_day_close)
            pivot_vals = [round(v, 2) for v in [pivot, r1, r2, r3, s1, s2, s3]]
        else:
            pivot_vals = [np.nan] * 7

        # Look up sentiment results
        sentiment_info = sentiment_df.loc[ticker_full] if ticker_full in sentiment_df.index else {'s_label': 'N/A', 'final_score': 0}

        signal_summary_df = pd.DataFrame({
            'stock': [ticker_full],
            'price': [last_price],
            's_label': [sentiment_info['s_label']],
            'news_#': [sentiment_info['final_score']],
            'EMA': [last_ema_signal],
            'MACD': [last_macd_signal],
            'Stoc': [last_stoch_signal],
            'RSI': [last_rsi_signal],
            'BB': [last_bb_signal],
            'MOM': [last_momentum_signal],
            'BBP': [last_bbp_signal],
            'OBV': [last_obv_signal],
            'VPT': [last_vpt_signal], # Add VPT to the summary
            'Pivot': [pivot_vals[0]], 'R1': [pivot_vals[1]], 'R2': [pivot_vals[2]], 'R3': [pivot_vals[3]],
            'S1': [pivot_vals[4]], 'S2': [pivot_vals[5]], 'S3': [pivot_vals[6]]
        })
        all_signal_summaries.append(signal_summary_df)

    except Exception as e:
        print(f"Error processing {ticker_full}: {e}")

# --- Step 3: Create the Final Report ---

if all_signal_summaries:
    final_report_df = pd.concat(all_signal_summaries, ignore_index=True)

    # Define weights for a more nuanced score, including VPT
    weights = {
        'EMA': ema_w,
        'MACD': macd_w,
        'Stoc': stoc_w,
        'RSI': rsi_w,
        'BB': bb_w,
        'MOM': mom_w,
        'BBP': bbp_w,
        'OBV': obv_w,
        'VPT': vpt_w, # Use the new weight for VPT
    }

    # Calculate a combined "Buy Score"
    final_report_df['Buy_#'] = 0
    for signal, weight in weights.items():
        final_report_df['Buy_#'] += final_report_df[signal].apply(lambda x: weight if x == 'BUY' else -weight if x == 'SELL' else 0)

    # Add sentiment to the score (a small weight)
    final_report_df['Buy_#'] += final_report_df['news_#'] * 2.0

    # Filter for positive sentiments only
    positive_sentiment_df = final_report_df[final_report_df['s_label'] == 'Positive'].copy()

    # Sort the report by the combined score
    positive_sentiment_df = positive_sentiment_df.sort_values(by='Buy_#', ascending=False)
    positive_sentiment_df = positive_sentiment_df.round(2)

    # Define columns for the final, consolidated report
    report_cols = [
        'stock', 'Buy_#', 'news_#',
        'EMA', 'MACD', 'Stoc', 'RSI', 'BB',
        'MOM', 'BBP', 'OBV', 'VPT',
        'price','R1', 'R2', 'R3',
        's_label','Pivot', 'S1', 'S2', 'S3'
    ]
    positive_sentiment_df = positive_sentiment_df[report_cols]

    print("\n\n--- Daily Stock Trading Report (Positive Sentiments Only) ---")
    print("A ranked list of stocks with positive sentiment, based on a combined sentiment and technical analysis score.")
    print("Higher scores indicate a stronger potential 'BUY' signal.")
    print("\nRecommended for Further Investigation (Top Stocks):")

    top_stocks = positive_sentiment_df.head(top_n)
    print(tabulate(top_stocks, headers='keys', tablefmt='grid'))

else:
    print("\nNo signal summaries were generated.")

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.3 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=18b4c0e8a985d410831bf786be80de751170cde1bf7b26c09d694fea27e73e64
  Stored in directory: /root/.cache/pip/wheels/03/f5/1a/23761066dac1d0e8e683e5fdb27e12de53209d05a4a37e6246
Successfully built sgmllib3k


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu

Processing Technical Data: 100%|████████████████████████████████|74/74 [00:37 <00:00,  1.98stock/s ]



--- Daily Stock Trading Report (Positive Sentiments Only) ---
A ranked list of stocks with positive sentiment, based on a combined sentiment and technical analysis score.
Higher scores indicate a stronger potential 'BUY' signal.

Recommended for Further Investigation (Top Stocks):
+----+---------+---------+----------+-------+--------+--------+-------+------+-------+-------+-------+-------+---------+--------+--------+--------+-----------+---------+--------+--------+--------+
|    | stock   |   Buy_# |   news_# | EMA   | MACD   | Stoc   | RSI   | BB   | MOM   | BBP   | OBV   | VPT   |   price |     R1 |     R2 |     R3 | s_label   |   Pivot |     S1 |     S2 |     S3 |
+====+=========+=========+==========+=======+========+========+=======+======+=======+=======+=======+=======+=========+========+========+========+===========+=========+========+========+========+
| 42 | BTE.to  |    6.54 |     1.37 | BUY   | ---    | ---    | ---   | ---  | BUY   | BUY   | BUY   | ---   |    2.95 |   2.